# Install Libraries

In [1]:
!pip install assemblyai
!pip install python-dotenv
#!pip install openai==0.28
#!pip install --upgrade openai
!pip uninstall openai -y
!pip install openai
!pip install speechrecognition
!pip install pyttsx3
!pip install pydub
!pip install ffprobe
!pip install ffmpeg

Found existing installation: openai 1.65.2
Uninstalling openai-1.65.2:
  Successfully uninstalled openai-1.65.2
  Using cached openai-1.65.2-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.65.2-py3-none-any.whl (473 kB)


# Importing Libarries

In [2]:
import assemblyai as aai
import linecache
import os
import glob
import textwrap
import speech_recognition as sr
from transformers import pipeline
import pyttsx3
import openai
from openai import OpenAI
from dotenv import load_dotenv
from pydub import AudioSegment
from pydub.utils import make_chunks
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


# API Key Setup

In [3]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
!cat /content/drive/MyDrive/assemblyai_api_key.txt >> .env

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ASSEMBLYAI_API_KEY = os.getenv("ASSEMBLYAI_API_KEY")

# Audio to Text

In [5]:
m4a_file = '/content/Positive-call.m4a'
wav_filename = 'Positive-call.wav'

sound = AudioSegment.from_file(m4a_file, format='m4a')
file_handle = sound.export(wav_filename, format='wav')

In [22]:
# Load the audio file
audio = AudioSegment.from_file('/content/Positive-call.wav', format='wav')

# Define chunk length in milliseconds (e.g., 30 seconds)
chunk_length_ms = 30000

# initialize the recognizer
r = sr.Recognizer()

# Split the audio into chunks
chunks = make_chunks(audio, chunk_length_ms)
text_list = []
# Process each chunk individually using SpeechRecognition
for i, chunk in enumerate(chunks):
    chunk_name = f"chunk{i}.wav"
    chunk.export(chunk_name, format="wav")

    # Now use SpeechRecognition on each chunk_name
    with sr.AudioFile(chunk_name) as source:
        # listen for the data (load audio to memory)
        audio_data = r.record(source)
        # recognize (convert from speech to text)
        try:  # Use try-except to handle potential errors
            text = r.recognize_google(audio_data)
            print(text)
            text = text.replace('\n', ' ')
            text_list.append(text)
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")

thank you for calling Chris Sean this is Candice how may I help you I still have not received my order you said I would receive it on the 20th so the 22nd still nothing I don't know what's going on but if it's not too much of a bother to you I would really really know love to get what I paid for you know what I mean if it's beyond the promise delivery date would definitely need to look into that may have your number and your full name so I can check for you that's exactly what I'm thinking
what's taking so long anyway the order number is 498-4977 and my name is Tabitha ratchet thank you Tabitha I will now go ahead and pull up your order and hopefully I can give you an immediate answer one moment please go ahead thank you so it's two days
normally when a person is late like this we send an email and for me of the delay so let me visit the FedEx website and track it thank you Candice that would be appreciated I am I actually haven't tried checking on the FedEx website since you already g

In [7]:
def combine_wav_files(file_paths, output_path):
    """Combines multiple WAV files into one.

    Args:
        file_paths (list): List of paths to the WAV files to combine.
        output_path (str): Path to save the combined WAV file.
    """
    combined_audio = AudioSegment.empty()
    for file_path in file_paths:
        audio = AudioSegment.from_wav(file_path)
        combined_audio += audio
    combined_audio.export(output_path, format="wav")

In [8]:
combine_wav_files(glob.glob('/content/chunk*.wav'), '/content/combined_audio.wav')

In [23]:
text_excerpt = ' '.join(text_list)
text_excerpt

"thank you for calling Chris Sean this is Candice how may I help you I still have not received my order you said I would receive it on the 20th so the 22nd still nothing I don't know what's going on but if it's not too much of a bother to you I would really really know love to get what I paid for you know what I mean if it's beyond the promise delivery date would definitely need to look into that may have your number and your full name so I can check for you that's exactly what I'm thinking what's taking so long anyway the order number is 498-4977 and my name is Tabitha ratchet thank you Tabitha I will now go ahead and pull up your order and hopefully I can give you an immediate answer one moment please go ahead thank you so it's two days normally when a person is late like this we send an email and for me of the delay so let me visit the FedEx website and track it thank you Candice that would be appreciated I am I actually haven't tried checking on the FedEx website since you already 

# Sentiment Analysis

In [24]:
sentiment_messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze and detect the sentiment of a conversation between customer and customer service agent."}
          ]
sentiment_prompt = f"Analyze the text excerpt {text_excerpt} and determine the sentiment of the conversation in a single word: Positive, Negative or Neutral"
sentiment_messages.append({"role": "user", "content": sentiment_prompt})
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

sentiment_analysis_chat_completion = client.chat.completions.create(
    messages=sentiment_messages,
    model="gpt-4o",
    temperature=0
)
print(sentiment_analysis_chat_completion.choices[0].message.content)

Negative


# Extract Key Topics

In [27]:
key_topic_messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze a conversation between customer and customer service agent. You help finding out the reason of the customer call."}
          ]

key_points_prompt = f"Analyze the text excerpt {text_excerpt} and determine the reason of the customer call in few words."
key_topic_messages.append({"role": "user", "content": key_points_prompt})
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

key_topic_chat_completion = client.chat.completions.create(
    messages=key_topic_messages,
    model="gpt-4o",
    temperature=0
)
print(key_topic_chat_completion.choices[0].message.content)
reason = key_topic_chat_completion.choices[0].message.content

The customer, Tabitha Ratchet, called to report a missing order that was supposed to be delivered by FedEx but was not received.


# Action Plan

In [28]:
action_plans_messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze a conversation between customer and customer service agent. You help finding out the action plans to be taken with respect to the customer call reason."}
          ]

action_plans_prompt = f"Analyze the text excerpt {text_excerpt} and {reason} and determine the action plans with respect to the customer call to resolve."
action_plans_messages.append({"role": "user", "content": action_plans_prompt})
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

action_plan_chat_completion = client.chat.completions.create(
    messages=action_plans_messages,
    model="gpt-4o",
    temperature=0
)
print(action_plan_chat_completion.choices[0].message.content)

Based on the conversation between Tabitha Ratchet and Candice, the customer service agent, the following action plans have been determined to resolve the issue of the missing order:

1. **Verify Delivery Information**: Candice checked the FedEx tracking information, which indicated that the parcel was delivered on the 20th at 10 a.m. However, Tabitha confirmed that the delivery details were incorrect, as her apartment has no front porch and all parcels should be delivered to the concierge.

2. **File a PDR Claim**: Candice will file a Parcel Delivered Not Received (PDR) claim with FedEx on behalf of Tabitha. This claim will initiate an investigation by FedEx to locate the missing parcel.

3. **Email Confirmation**: Candice sent an email to Tabitha, requesting her to confirm that she has not received the parcel. Tabitha needs to reply to this email to provide documentation for the PDR claim.

4. **Investigation Timeline**: The investigation by FedEx is expected to take 5 to 7 business d